In [17]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [18]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [19]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [20]:
##GPU 사용 시
device = torch.device("cuda:0")

In [21]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [22]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset("ko_data.tsv",encoding="CP949", field_indices=[0,1], num_discard_samples=1)

In [24]:
print(dataset_train[0])
print(dataset_test[0])
print(len(dataset_train))
print(len(dataset_test))

['아 더빙.. 진짜 짜증나네요 목소리', '0']
['굳 ㅋ', '1']
150000
50000


In [25]:
dataset = nlp.data.ConcatDataset([dataset_train, dataset_test])

In [26]:
len(dataset)

200000

In [27]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [28]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [34]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-5

In [29]:
data_train = BERTDataset(dataset, 0, 1, tok, max_len, True, False)
print(data_train[0])

(array([   2, 3093, 1698, 6456,  517,   54,  517,   54, 4368, 4396, 7316,
       5655, 5703, 2073,    3,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32), array(15, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32), 0)


In [ ]:
(array([   2, 3093, 1698, 6456,  517,   54,  517,   54, 4368, 4396, 7316,
       5655, 5703, 2073,    3,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32), 
 array(15, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32), 0)

In [31]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [57]:
len(train_dataloader)

3125

In [32]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [35]:
model = BERTClassifier(bertmodel, dr_rate=0.1).to(device)

In [36]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [37]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [38]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [39]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [40]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [64]:
lst = []
i = 1
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
    if i != 1:
        break
    sample_val = [token_ids, valid_length, segment_ids, label]
    print([token_ids, valid_length, segment_ids, label])
    i += 1
    

[tensor([[   2, 3093, 1698,  ...,    1,    1,    1],
        [   2,  517, 7989,  ...,    1,    1,    1],
        [   2, 1458, 7191,  ...,    1,    1,    1],
        ...,
        [   2, 2485, 6821,  ...,    1,    1,    1],
        [   2, 4841, 6333,  ...,    1,    1,    1],
        [   2, 2355, 5842,  ...,    1,    1,    1]], dtype=torch.int32), tensor([15, 36, 14, 26, 47, 41, 16, 64, 13, 38, 17, 35, 29, 34, 34, 13, 45, 23,
        35, 27, 27, 10, 64, 17, 11, 42, 19,  9,  6, 36, 33, 25, 12, 24, 14, 15,
        23, 30, 14, 12, 11, 57, 19, 30, 59, 17, 64, 38, 22, 64, 32, 32, 18, 41,
         4,  5,  3, 64,  8, 11, 14, 23, 26, 56], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 

In [69]:
print(np.shape(sample_val[0]))
print(np.shape(sample_val[1]))
print(np.shape(sample_val[2]))
print(np.shape(sample_val[3]))

torch.Size([64, 64])
torch.Size([64])
torch.Size([64, 64])
torch.Size([64])


In [ ]:
[tensor([[   2, 3093, 1698,  ...,    1,    1,    1],
        [   2,  517, 7989,  ...,    1,    1,    1],
        [   2, 1458, 7191,  ...,    1,    1,    1],
        ...,
        [   2, 2485, 6821,  ...,    1,    1,    1],
        [   2, 4841, 6333,  ...,    1,    1,    1],
        [   2, 2355, 5842,  ...,    1,    1,    1]], dtype=torch.int32), 
 tensor([15, 36, 14, 26, 47, 41, 16, 64, 13, 38, 17, 35, 29, 34, 34, 13, 45, 23,
        35, 27, 27, 10, 64, 17, 11, 42, 19,  9,  6, 36, 33, 25, 12, 24, 14, 15,
        23, 30, 14, 12, 11, 57, 19, 30, 59, 17, 64, 38, 22, 64, 32, 32, 18, 41,
         4,  5,  3, 64,  8, 11, 14, 23, 26, 56], dtype=torch.int32), 
 tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), 
 tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0], dtype=torch.int32)]

In [41]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
#     model.eval()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         test_acc += calc_accuracy(out, label)
#     print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    torch.save(model, "ko_model_save/ko_model_save_acc_06_22" + str(test_acc/ (batch_id+1)))

/home/key13689/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/opt/conda/conda-bld/pytorch_1587428266983/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


epoch 1 batch id 1 loss 0.6795390248298645 train acc 0.59375


KeyboardInterrupt: 

In [ ]:
def convert_input_data(sentence):
    transform = nlp.data.BERTSentenceTransform(
            tok, max_seq_length=max_len, pad=True, pair=False)
    return transform(sentence)

In [ ]:
def test_sentences(sentences):
    model.eval()
    inputs = convert_input_data(sentence)
    with torch.no_grad():
        outputs = model 

In [21]:
def model_test(path):
    test_acc = 0.0
    model = torch.load(path)
    model.eval()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("test acc {}".format(test_acc / (batch_id+1)))

In [ ]:
model_test("ko_model_save/ko_model_save_acc_06_160.8971787084398977")

/home/key13689/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """
